In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import roc_auc_score, f1_score
from scipy.stats import entropy
from gensim.models import Word2Vec
import time
import gc
pd.set_option('display.max_columns', None)

import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr_torch.models import *
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
import torch
from tqdm import tqdm as tq

df = reduce_mem(df)


def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df




In [ ]:
labels = joblib.load("labels.pkl")# train label
df = joblib.load("df.pkl")        # train,test concat
train_num = joblib.load("train_num.pkl")


# 融合gbdt，input->多个神经元->cancat->batch_Norm

In [ ]:
meta_oofLIST = []
oof_cnt = 0
import os
test_num = len(df) - train_num
for i in os.listdir("stacking/stacking/"):
    if i[-3:] != "pkl":
        continue
    pred = pd.read_pickle("stacking/stacking/"+i)
    oof = pred["oof"]
    testpred = pred["pred"]
    if len(oof) == train_num:
        oof_cnt += 1
        oof = list(oof) + list(testpred)
        df["oof_"+str(oof_cnt)] = oof
        meta_oofLIST.append("oof_"+str(oof_cnt))
print(meta_oofLIST)

# 读取zhangqi的oof，下面的模型没用到，即下面是一个单模型，显然也可以融合进去

# 融合deviceid_newsid_deepwalk_8

In [ ]:

deepwalk = pd.read_pickle("deviceid_newsid_deepwalk_8.pkl")

t1 = pd.read_csv('dataset/train.csv')
t2 = pd.read_csv('dataset/test.csv')

DEVICE = list(t1["deviceid"]) + list(t2["deviceid"])

df["deviceid"] = DEVICE
del t1,t2
df = df.merge(deepwalk,how='left',on='deviceid')

DFCOLUMNS = list(df.columns)
deepwalk.head()

# 统计各类特征的名称

In [ ]:

from collections import *

exposure_ts_gapLIST = [i for i in DFCOLUMNS if i[-3:]=='gap']
exposure_ts_gapLIST
countLIST = [i for i in DFCOLUMNS if 'count' in i]
countLIST
categoryLIST = [
    'deviceid', 'newsid', 'pos', 'app_version', 'device_vendor',
    'netmodel', 'osversion', 'device_version', 'lng', 'lat', 'lng_lat',
    'day','hour','minute'
]
embLIST = [i for i in DFCOLUMNS if 'emb' in i]
embLIST
nuniqueLIST = [i for i in DFCOLUMNS if 'nunique' in i]
nuniqueLIST


ctrLIST = [i for i in DFCOLUMNS if 'ctr'in i]

otherLIST =  [i for i in DFCOLUMNS if i not in exposure_ts_gapLIST and\
              i not in countLIST and i not in embLIST and i not in nuniqueLIST\
             and i not in categoryLIST and i not in ctrLIST]
otherLIST


# small_exposure_ts_gapLIST = defaultdict(list)
# for i in exposure_ts_gapLIST:
#     t = i.split("exposur")[0][:-2]
#     small_exposure_ts_gapLIST[t].append(t)
#     print(t)


LISTmap = {
    "exposure_ts_gapLIST":exposure_ts_gapLIST,
    "countLIST":countLIST,
    "otherLIST":otherLIST,
    "embLIST":embLIST,
    "categoryLIST":categoryLIST,
    "nuniqueLIST":nuniqueLIST,
#     "meta_oofLIST":meta_oofLIST
}


sparse_features = categoryLIST
dense_features = []
for i in LISTmap:
    if i != "categoryLIST" :#and i != "countLIST":
        dense_features += LISTmap[i]
print(len(dense_features))

## 获取各组指标的索引

In [ ]:
'gap' in "lng_lat_next2_exposure_ts_gap"
# print(meta_oofLIST)
train_num == len(labels)
from collections import *
featureNum = []

indexMap = defaultdict(list)
for index,i in enumerate(dense_features):
    for j in LISTmap:
        if j == "categoryLIST":
            continue
        if i in LISTmap[j]:
            indexMap[j].append(index)
input_dim = 0

for i in indexMap:
    print(i,len(indexMap[i]))
    if i != "meta_oofLIST":
        featureNum.append(len(indexMap[i]))
        input_dim += len(indexMap[i])
print(input_dim)
featureNum


In [ ]:


sampleRate = 1 


# 过滤异常值，有些值太大时，归一化后，使得数据整体偏小，不利于NN学习

for i in LISTmap["exposure_ts_gapLIST"]:
    q = df[i].quantile(0.5)
    print(i,q)
    df.loc[df[i]>q,i] = q

df[dense_features] = df[dense_features].fillna(0)

dfmin = np.min(df[dense_features])
dfmax = np.max(df[dense_features])

dfdiff = dfmax - dfmin 
dfdiff[dfdiff==0] = 1


print("计算min、max完成")

data = df[:train_num][::sampleRate]
test = df[train_num:][::sampleRate]





target = ['target']

data["target"] = labels[:train_num][::sampleRate]
test["target"] = [0]*len(test)


In [ ]:
gc.collect()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torch.optim.optimizer import Optimizer
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import lightgbm as lgb
from sklearn.metrics import mean_squared_error

import os
class CyclicLR(object):
    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)
        
        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration
    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs
class groupy_NN(nn.Module):
    def __init__(self ,input_dim ,hidden_dim, dropout = 0.75):
        super(groupy_NN, self).__init__()
        
        self.inpt_dim = input_dim
        self.hidden_dim = hidden_dim
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.concat_dim = input_dim * 2 #+ len(indexMap["meta_oofLIST"]) * 10
        
        self.sub_meta = nn.Sequential(nn.Linear(1,10),nn.ReLU(),nn.Linear(10,10),nn.ReLU())
        self.subfc0 = nn.Sequential(nn.Linear(featureNum[0],featureNum[0]*2),nn.ReLU(),nn.Linear(featureNum[0]*2,featureNum[0]*2),nn.ReLU())
        self.subfc1 = nn.Sequential(nn.Linear(featureNum[1],featureNum[1]*2),nn.ReLU(),nn.Linear(featureNum[1]*2,featureNum[1]*2),nn.ReLU())
        self.subfc2 = nn.Sequential(nn.Linear(featureNum[2],featureNum[2]*2),nn.ReLU(),nn.Linear(featureNum[2]*2,featureNum[2]*2),nn.ReLU())
        self.subfc3 = nn.Sequential(nn.Linear(featureNum[3],featureNum[3]*2),nn.ReLU(),nn.Linear(featureNum[3]*2,featureNum[3]*2),nn.ReLU())
        self.subfc4 = nn.Sequential(nn.Linear(featureNum[4],featureNum[4]*2),nn.ReLU(),nn.Linear(featureNum[4]*2,featureNum[4]*2),nn.ReLU())
        
        
        self.fc1 = nn.Linear(self.concat_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim//2)
        self.fc_tmp = nn.Sequential(nn.Linear(hidden_dim//2, hidden_dim//2),nn.ReLU())
        self.fc3 = nn.Linear(hidden_dim//2, 1)
        
        self.bn1 = nn.BatchNorm1d(self.concat_dim)
        
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.bn3 = nn.BatchNorm1d(int(hidden_dim/4))
        self.bn4 = nn.BatchNorm1d(int(hidden_dim/8))
    
    def forward(self, x):
        b_size = x.size(0)
#         print(x.size)
        groupyX = []
        for i in indexMap:
            if i != "meta_oofLIST":
                groupyX.append(x[:,indexMap[i]])
    
        meta_X = []
#         print(len(indexMap["meta_oofLIST"]))
#         for i in indexMap["meta_oofLIST"]:
#             meta_X.append(x[:,[i]])

        embX = []
        for i in meta_X:
            embX.append(self.sub_meta(i))
    
#         embX_sub = torch.cat(embX,axis=1)
#         print(embX_sub.size())
        selfFCLIST = [self.subfc0,self.subfc1,self.subfc2,self.subfc3,self.subfc4]
        
        for index,i in enumerate(selfFCLIST):
            embX.append(selfFCLIST[index](groupyX[index]))
        
        
        out_emb = torch.cat(embX,axis=1)
        out_emb = self.bn1(out_emb)
        
#         x = x.view(-1, 1)
        y = self.fc1(out_emb)
        y = self.relu(y)
        y = self.bn2(y)
        y = self.fc2(y)
        y = self.relu(y)
        y = self.fc_tmp(y)
        out= self.fc3(y)
        return out
class TorchDataset(Dataset):
    def __init__(self, df):
        self.df = df
        
        pass
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        label = self.df.loc[i,"target"]
        data = self.df.loc[i,dense_features].copy()# - dfmin)/dfdiff
        data = (data - dfmin) / dfdiff
        return torch.tensor(data,dtype=torch.float32).cuda(),\
    torch.tensor([label], dtype=torch.float32).cuda()
    


In [ ]:
# data[dense_features].info()

In [ ]:
# test.describe()
len(dense_features)

**train val split()**


In [ ]:
n_epochs = 2
batch_size = 1024 * 4
test_size = batch_size 
Test = TorchDataset(test.reset_index())
test_loader = torch.utils.data.DataLoader(Test, batch_size=test_size, shuffle=False)

print("len(test_loader):",len(test_loader))


# train, val = train_test_split(data.iloc[::10], test_size=0.2)

# Train = data[data["day"]<10].iloc[::10]
val = data[data["day"]==10].iloc[::10]
Train = data.iloc[::5]

train = TorchDataset(Train.reset_index())
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

print("len(train_loader):",len(train_loader))


Val = TorchDataset(val.reset_index())
val_loader = torch.utils.data.DataLoader(Val, batch_size=batch_size, shuffle=False)

loss_fn = torch.nn.BCEWithLogitsLoss()
model = groupy_NN(input_dim,1000)


test_preds = np.zeros((len(test)))


print(model)
model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4,weight_decay=1e-5) # Using Adam optimizer
def sigmoid(x):
    return 1 / (1 + np.exp(-x))



for epoch in range(n_epochs):
    print("_______________________",epoch,"_________________________")
    start_time = time.time()
    model.train()
    avg_loss = 0.
    #avg_auc = 0.
    test_preds_fold = np.zeros((len(test)))
    valid_preds_fold = np.zeros((len(val)))
    for i, (x_batch, y_batch) in enumerate(train_loader):
        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)

        if i%10 == 0:
            print("BATCH_{}_loss_________________________{}".format(i,round(loss.item(),4)))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item()/len(train_loader)
    model.eval()
    avg_val_loss = 0.
    for i, (x_batch, y_batch) in enumerate(val_loader):
        
        y_pred = model(x_batch).detach()
        if i%20 == 0:
            print(sigmoid(y_pred.cpu().numpy())[:,0])
        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(val_loader)
        valid_preds_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
            
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
        epoch + 1, n_epochs, avg_loss, avg_val_loss, elapsed_time))
    auc = round(roc_auc_score(val["target"].values,valid_preds_fold),4)
    print(auc)
    model.eval()


In [ ]:

model.eval()
print("test")
for i, (x_batch,_) in enumerate(test_loader):
    y_pred = model(x_batch).detach()
#         print(x_batch.size())
    test_preds_fold[i * test_size:(i+1) * test_size] = sigmoid(y_pred.cpu().numpy())[:,0]

    if i % 20==0:
        print("test______",i)
        print(sigmoid(y_pred.cpu().numpy())[:,0])
#             print(y_pred.cpu().numpy().shape)
#     print(auc)
test_preds += test_preds_fold 


sub = pd.read_csv('dataset/sample.csv')
sub["target"] = test_preds_fold
print('=============================================== sub save ===============================================')
sub.to_csv('sub_prob_{}_{}_{}.csv'.format(auc, best_f1, sub['target'].mean()), index=False)
sub['target'] = sub['target'].apply(lambda x: 1 if x >= best_t else 0)
sub.to_csv('sub_{}_{}_{}.csv'.format(auc, best_f1, sub['target'].mean()), index=False)
# print('runtime:', time.time() - t)
print('finish.')
print('========================================================================================================')

     